In [2]:
%load_ext autoreload
%autoreload 2
%cd ..
import numpy as np
import matplotlib.pyplot as plt
from src.ortools_solver import CVRP_solver
from src.utils import distance_calculator, calculate_distance_matrix, show_matrix, print_command, calculate_num_rows
import pygmtools as pygm
import os, shutil

/home/yq-mew/yq/chem1906/Projects/pipette_scheduling


/u/yq/chem1906/anaconda3/envs/pipette_scheduling/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def calculate_S_E(a, P=None):
    # a the jobs matrix, row is the source, column is the destination
    if P is not None:
        a = np.dot(a, P)
    a[a>0] = 1
    jobs = np.argwhere(a)
    starting_point = np.zeros((jobs.shape[0], a.shape[0]))
    ending_point = np.zeros((jobs.shape[0], a.shape[1]))
    ending_map = dict(np.argwhere(P if P is not None else np.eye(a.shape[1])).tolist())
    new_jobs = []
    for i in range(jobs.shape[0]):
        starting_point[i, jobs[i, 0]] = 1
        ending_point[i, ending_map[jobs[i, 1]]] = 1
        new_jobs.append([jobs[i, 0], ending_map[jobs[i, 1]]])
    if P is None:
        return starting_point, ending_point
    else:
        return starting_point, ending_point, np.array(new_jobs)

def calculate_D_prime(D_S, D_D, S, E):
    # calculate D' = S(D_S)S^T + E(D_D)E^T
    D_prime = np.dot(np.dot(S, D_S), S.T) + np.dot(np.dot(E, D_D), E.T)
    return D_prime

def add_depot(D_prime):
    # add a depot to D'
    D_prime = np.vstack((np.zeros(D_prime.shape[0]), D_prime))
    D_prime = np.hstack((np.zeros((D_prime.shape[0], 1)), D_prime))
    return D_prime

def get_optimized_sequence(recorder):
    # get the optimized sequences from the VCRP solver, pad with -1 and sort
    for i in range(len(recorder)):
        recorder[i] = np.array(recorder[i])
        recorder[i] = np.pad(recorder[i], (0, 8-recorder[i].shape[0]), 'constant', constant_values=-1)
    # move the elements containing -1 to the end
    optimized_seuqnece = np.array(recorder)
    optimized_seuqnece = np.array(sorted(optimized_seuqnece, key=lambda x: np.sum(x!=-1)))
    optimized_seuqneces = np.array(optimized_seuqnece[::-1])
    return optimized_seuqneces

def calculate_T(sequences):
    # the matrix should be paddled with -1, return a n*n matrix
    # sequences is a n*8 matrix

    sequences_flat = sequences.flatten()
    sequences_flat = sequences_flat[sequences_flat != -1]
    zeros = np.zeros((sequences_flat.shape[0],sequences_flat.shape[0]))
    for sequence in sequences:
        for i in range(sequence.shape[0]-1):
            if sequence[i] != -1 and sequence[i+1] != -1:
                zeros[sequence[i]-1,sequence[i+1]-1] = 1
            else:
                break
    return zeros

In [4]:
def calculate_D(labware:int):
    num_rows = calculate_num_rows(labware)
    D = np.ones((labware,labware))
    for i in range(labware):
        for j in range(labware):
            if i//num_rows == j//num_rows:
                if i-j == -1:
                    D[i, j] = 0

    if labware == 384:
        D = np.ones((labware,labware))
        for i in range(labware):
            for j in range(labware):
                if i//num_rows == j//num_rows:
                    if i-j == -2:
                        D[i, j] = 0
    return D

In [ ]:
def calculate_T(sequences):
    # the matrix should be paddled with -1, return a n*n matrix
    # sequences is a n*8 matrix

    sequences_flat = sequences.flatten()
    sequences_flat = sequences_flat[sequences_flat != -1]
    zeros = np.zeros((sequences_flat.shape[0],sequences_flat.shape[0]))
    for sequence in sequences:
        for i in range(sequence.shape[0]-1):
            if sequence[i] != -1 and sequence[i+1] != -1:
                zeros[sequence[i]-1,sequence[i+1]-1] = 1
            else:
                break
    return zeros

def calculate_cost(task_matrix):
    # T is a n*n matrix
    # D_prime is a (n+1)*(n+1) matrix with the zero paddle
    # return the cost of the T and D_prime
    jobs = np.argwhere(task_matrix)
    D_S = calculate_D(task_matrix.shape[0])
    D_D = calculate_D(task_matrix.shape[1])   
    S, E = calculate_S_E(task_matrix)
    D_prime = calculate_D_prime(D_S,D_D, S, E)
    
    t = calculate_T(jobs)
    # cost = trace (T^T * D_prime)
    cost = np.trace(np.dot(t.T, D_prime))
    return cost

In [6]:
def CVRP_QAP(task_matrix, iteration=5, inner_cvrp_timewall=2, final_cvrp_timewall=10, ipfp_maxiter=50):
    task_matrix[task_matrix>0] = 1
    jobs = np.argwhere(task_matrix)
    D_S = calculate_D(task_matrix.shape[0])
    D_D = calculate_D(task_matrix.shape[1])   

    output_P = np.eye(task_matrix.shape[1])
    S, E = calculate_S_E(task_matrix)
    D_prime = calculate_D_prime(D_S,D_D, S, E)
    best_cost = float('inf')
    for i in range(iteration):
        # construct & update CVRP
        D_prime = add_depot(D_prime)

        # solve CVRP
        optimized_distance, recorder = CVRP_solver(D_prime.astype(np.int64), solving_time=inner_cvrp_timewall)

        optimized_seuqnecess = get_optimized_sequence(recorder)
        t = calculate_T(optimized_seuqnecess)
        D_prime = D_prime[1:, 1:]
        # cost = trace (T^T * D_prime)
        cost = np.trace(np.dot(t.T, D_prime))
        print(f'iter={i}, cost={cost} after CVRP')
        if best_cost > cost:
            best_cost = cost
            best_output_P = output_P

        # construct QAP
        A = np.dot(np.dot(E.T, t.T), E)
        B = D_D
        K = np.kron(1-B, A.T) # transform minimization into maximization

        # solve QAP
        P = pygm.ipfp((K + K.T), n1=task_matrix.shape[1], n2=task_matrix.shape[1], x0=np.eye(task_matrix.shape[1])[None,:,:], max_iter=ipfp_maxiter)

        # new_E = E * P
        new_E = np.dot(E, P)
        new_D_prime = calculate_D_prime(D_S,D_D, S, new_E)
        cost = np.trace(np.dot(t.T, new_D_prime))
        output_P = np.dot(output_P, P)
        print(f'iter={i}, cost={cost} after QAP')
        if best_cost > cost:
            best_cost = cost
            best_output_P = output_P

        # update params
        D_prime = new_D_prime
        E = new_E

    # calculate best CVRP result with more solving time
    S, E, new_jobs = calculate_S_E(task_matrix, best_output_P)
    D_prime = calculate_D_prime(D_S,D_D, S, E)
    D_prime = np.vstack((np.zeros(D_prime.shape[0]), D_prime))
    D_prime = np.hstack((np.zeros((D_prime.shape[0], 1)), D_prime))
    best_cost, best_recorder = CVRP_solver(D_prime.astype(np.int64), solving_time=final_cvrp_timewall)
    print(f'solution cost={best_cost}')

    # transform to job id sequence
    best_recorder = get_optimized_sequence(best_recorder)

    return best_cost, best_output_P, new_jobs, best_recorder

In [7]:
def random_choice(total_elements, chosen_elements):
    '''
    total_elements: total number of elements in the array
    chosen_elements: number of elements to be chosen
    '''
    a = np.zeros(total_elements)
    random_vector = np.random.rand(chosen_elements)
    random_vector = random_vector.round(2)
    random_vector = random_vector / random_vector.sum(axis=0, keepdims=1)
    a[:chosen_elements] = random_vector
    np.random.shuffle(a)
    return a

def random_choose_candidate(source_dim,dest_dim,non_zeros_dim): 
    '''
    num_candidate: number of candidate to be chosen
    total_candidate: total number of candidates
    chosen_elements: number of elements to be chosen
    '''
    # repeat the random_choice function for num_candidate times
    a = np.zeros((source_dim,dest_dim))
    for i in range(source_dim):
        candidate = random_choice(dest_dim,non_zeros_dim)
        a[i,:] = candidate
    return a.round(2)

In [15]:
labware_list =[12]
# enumerate all the two combinations of the labware_list
labware_combinations = []
for i in range(len(labware_list)):
    for j in range(len(labware_list)):
        labware_combinations.append([labware_list[i], labware_list[j]])

stats = []
np.random.seed(0)
for labware_combination in labware_combinations:
        print(f'labware_combination: {labware_combination}')
        for r in range (3):
            print('repeat:',r)
            for i in range(1,12,3):
                print(f'num_candidates={i}')
                experiments = random_choose_candidate(labware_combination[0],labware_combination[1],i)
                experiments[experiments>0] = 1
                jobs = np.argwhere(experiments)
                D_S = calculate_D(experiments.shape[0])
                D_D = calculate_D(experiments.shape[1])
                S, E = calculate_S_E(experiments)   
                # calculate distance matrix
                D_prime = calculate_D_prime(D_S,D_D, S, E)
                D_prime = add_depot(D_prime)
                # VRP solver
                VRP_distance, VRP_recorder = CVRP_solver(D_prime.astype(np.int64), solving_time =20)
                # calculate the cost of the non-optimized sequence
                tasks = np.array(range(jobs.shape[0]))
                tasks = tasks+1
                # if tasks.shape[0] %8 != 0, pad with -1
                if tasks.shape[0] %8 != 0:
                    tasks = np.pad(tasks, (0, 8-tasks.shape[0]%8), 'constant', constant_values=-1)
                unoptimized_seuqnece = tasks.reshape(-1, 8)
                t = calculate_T(unoptimized_seuqnece)
                d = D_prime[1:, 1:]
                non_optimized_distance = np.trace(np.dot(t.T, d))
                # change non_optimized_distance to integer
                non_optimized_distance = int(non_optimized_distance)

                # print the optimized sequence
                recorder = get_optimized_sequence(VRP_recorder)
                sequence = recorder.flatten()
                sequence = sequence[sequence!=-1] -1
                command_line = print_command(sequence, jobs,f'source{labware_combination[0]}', f'dest{labware_combination[1]}')
                np.savetxt(f'analysis/cost_analysis/optimized/optimized_command_line_cost_{VRP_distance}_s{labware_combination[0]}_d{labware_combination[1]}_r{r}.csv',
                           command_line,fmt='%s',delimiter=',')
                unoptimized_command_line = print_command(np.array(range(jobs.shape[0])),jobs,f'source{labware_combination[0]}', f'dest{labware_combination[1]}')
                np.savetxt(f'analysis/cost_analysis/unoptimized/unoptimized_command_line_cost_{non_optimized_distance}_s{labware_combination[0]}_d{labware_combination[1]}_r{r}.csv',
                           unoptimized_command_line,fmt='%s',delimiter=',')

labware_combination: [12, 12]
repeat: 0
num_candidates=1
num_candidates=4
num_candidates=7
num_candidates=10
repeat: 1
num_candidates=1
num_candidates=4
num_candidates=7
num_candidates=10
repeat: 2
num_candidates=1


/tmp/ipykernel_3259255/316573983.py:9: RuntimeWarning: invalid value encountered in divide
  random_vector = random_vector / random_vector.sum(axis=0, keepdims=1)


num_candidates=4
num_candidates=7
num_candidates=10


In [8]:
# random shuffle the unoptimized_command_line
labware_list =[96]
# enumerate all the two combinations of the labware_list
labware_combinations = []
for i in range(len(labware_list)):
    for j in range(len(labware_list)):
        labware_combinations.append([labware_list[i], labware_list[j]])

stats = []
np.random.seed(0)
for labware_combination in labware_combinations:
        print(f'labware_combination: {labware_combination}')
        for r in range (3):
            print('repeat:',r)
            for i in range(1,12,3):
                print(f'num_candidates={i}')
                experiments = random_choose_candidate(labware_combination[0],labware_combination[1],i)
                experiments[experiments>0] = 1
                jobs = np.argwhere(experiments)
                D_S = calculate_D(experiments.shape[0])
                D_D = calculate_D(experiments.shape[1])
                S, E = calculate_S_E(experiments)   
                # calculate distance matrix
                D_prime = calculate_D_prime(D_S,D_D, S, E)
                D_prime = add_depot(D_prime)
                # VRP solver
                VRP_distance, VRP_recorder = CVRP_solver(D_prime.astype(np.int64), solving_time =20)
                # calculate the cost of the non-optimized sequence
                tasks = np.array(range(jobs.shape[0]))
                tasks = tasks+1
                np.random.shuffle(tasks)
                # save it for the output of the unoptimized command line
                unoptimized_output = tasks -1
                # if tasks.shape[0] %8 != 0, pad with -1
                if tasks.shape[0] %8 != 0:
                    tasks = np.pad(tasks, (0, 8-tasks.shape[0]%8), 'constant', constant_values=-1)
                unoptimized_seuqnece = tasks.reshape(-1, 8)
                t = calculate_T(unoptimized_seuqnece)
                d = D_prime[1:, 1:]
                non_optimized_distance = np.trace(np.dot(t.T, d))
                # change non_optimized_distance to integer
                non_optimized_distance = int(non_optimized_distance)

                # print the optimized sequence
                recorder = get_optimized_sequence(VRP_recorder)
                sequence = recorder.flatten()
                sequence = sequence[sequence!=-1] -1
                command_line = print_command(sequence, jobs,f'source{labware_combination[0]}', f'dest{labware_combination[1]}')
                np.savetxt(f'analysis/cost_analysis/optimized/optimized_command_line_cost_{VRP_distance}_s{labware_combination[0]}_d{labware_combination[1]}_r{r}.csv',
                           command_line,fmt='%s',delimiter=',')
                unoptimized_command_line = print_command(unoptimized_output,jobs,f'source{labware_combination[0]}', f'dest{labware_combination[1]}')
                np.savetxt(f'analysis/cost_analysis/unoptimized/unoptimized_command_line_cost_{non_optimized_distance}_s{labware_combination[0]}_d{labware_combination[1]}_r{r}.csv',
                           unoptimized_command_line,fmt='%s',delimiter=',')

labware_combination: [96, 96]
repeat: 0
num_candidates=1
num_candidates=4
num_candidates=7
num_candidates=10
repeat: 1
num_candidates=1
num_candidates=4
num_candidates=7
num_candidates=10
repeat: 2
num_candidates=1
num_candidates=4
num_candidates=7
num_candidates=10


In [9]:
# get the name of the files in the directory analysis/cost_analysis
import os
import glob
import pandas as pd
names = []
for filename in glob.glob('analysis/cost_analysis/unoptimized/*.csv'):
    # get the name of the file without the path
    filename = os.path.basename(filename)
    names.append(filename)

# creat a dataframe to store the data
names_df = pd.DataFrame(names, columns=['filename'])
# save the dataframe to a csv file
names_df.to_csv('analysis/cost_analysis/unoptimized/filenames_mapping.csv', index=True)

In [10]:
import shutil
# create a subfolder called command_lines
if not os.path.exists('analysis/cost_analysis/command_lines'):
    os.makedirs('analysis/cost_analysis/command_lines')

# copy the files to the command_lines folder, rename the files as the index of the dataframe
for i in range(len(names_df)):
    # copy the file to the command_lines folder
    shutil.copy(f'analysis/cost_analysis/unoptimized/{names_df.iloc[i,0]}', f'analysis/cost_analysis/command_lines/{i}.csv')
    # rename the file as the index of the dataframe


In [22]:
names_df

,filename
0,0.csv
1,1.csv
2,2.csv
3,3.csv
4,4.csv
...,...
115,115.csv
116,116.csv
117,117.csv
118,118.csv


In [ ]:
experiments = random_choose_candidate(96,96,10)
experiments[experiments>0] = 1
jobs = np.argwhere(experiments)
row_num =0
sorted_jobs = []

D_S = calculate_D(experiments.shape[0])
D_D = calculate_D(experiments.shape[1])
S, E = calculate_S_E(experiments)   
# calculate distance matrix
D_prime = calculate_D_prime(D_S,D_D, S, E)
D_prime = add_depot(D_prime)
optimized_distance = 0
VRP_distance, recorder = CVRP_solver(D_prime.astype(np.int64), solving_time =20)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
tasks = np.array(range(jobs.shape[0]))
tasks = tasks+1
# if tasks.shape[0] %8 != 0, pad with -1 until the length is 8
if tasks.shape[0] %8 != 0:
    tasks = np.pad(tasks, (0, 8-tasks.shape[0]%8), 'constant', constant_values=-1)
unoptimized_seuqnece = tasks.reshape(-1, 8)
t = calculate_T(unoptimized_seuqnece)

In [ ]:
non_optimized_distance = distance_calculator(jobs)
non_optimized_distance 

In [ ]:
sequence = recorder.flatten()
sequence = sequence[sequence!=-1] -1
command_line = print_command(sequence, new_jobs)
np.savetxt('optimized_command_line.csv',command_line,fmt='%s',delimiter=',')

In [ ]:
jobs = np.argwhere(a)
unoptimized_command_line = print_command(np.array(range(jobs.shape[0])),jobs)
np.savetxt('unoptimized_command_line.csv',unoptimized_command_line,fmt='%s',delimiter=',')

In [ ]:
labware_list =[12,24,384]
# enumerate all the two combinations of the labware_list
labware_combinations = []
for i in range(len(labware_list)):
    for j in range(len(labware_list)):
        labware_combinations.append([labware_list[i], labware_list[j]])

stats = []
np.random.seed(0)
for labware_combination in labware_combinations:
        print(f'labware_combination: {labware_combination}')
        for r in range (3):
            print('repeat:',r)
            for i in range(1,12,3):
                print(f'num_candidates={i}')
                experiments = random_choose_candidate(labware_combination[0],labware_combination[1],i)
                experiments[experiments>0] = 1
                jobs = np.argwhere(experiments)
                D_S = calculate_D(experiments.shape[0])
                D_D = calculate_D(experiments.shape[1])
                S, E = calculate_S_E(experiments)   
                # calculate distance matrix
                D_prime = calculate_D_prime(D_S,D_D, S, E)
                D_prime = add_depot(D_prime)
                # VRP solver
                VRP_distance, VRP_recorder = CVRP_solver(D_prime.astype(np.int64), solving_time =20)
                # calculate the cost of the non-optimized sequence
                tasks = np.array(range(jobs.shape[0]))
                tasks = tasks+1
                # if tasks.shape[0] %8 != 0, pad with -1
                if tasks.shape[0] %8 != 0:
                    tasks = np.pad(tasks, (0, 8-tasks.shape[0]%8), 'constant', constant_values=-1)
                unoptimized_seuqnece = tasks.reshape(-1, 8)
                t = calculate_T(unoptimized_seuqnece)
                d = D_prime[1:, 1:]
                non_optimized_distance = np.trace(np.dot(t.T, d))
                # change non_optimized_distance to integer
                non_optimized_distance = int(non_optimized_distance)

                # print the optimized sequence
                recorder = get_optimized_sequence(VRP_recorder)
                sequence = recorder.flatten()
                sequence = sequence[sequence!=-1] -1

                if not os.path.exists(f'analysis/cost_analysis/optimized/{labware_combination[0]}_{labware_combination[1]}'):
                    os.makedirs(f'analysis/cost_analysis/optimized/{labware_combination[0]}_{labware_combination[1]}')
                command_line = print_command(sequence, jobs,f'source{labware_combination[0]}', f'dest{labware_combination[1]}')
                np.savetxt(f'analysis/cost_analysis/optimized/{labware_combination[0]}_{labware_combination[1]}/optimized_command_line_cost_{VRP_distance}_s{labware_combination[0]}_d{labware_combination[1]}_r{r}.csv',
                           command_line,fmt='%s',delimiter=',')
                if not os.path.exists(f'analysis/cost_analysis/unoptimized/{labware_combination[0]}_{labware_combination[1]}'):
                    os.makedirs(f'analysis/cost_analysis/unoptimized/{labware_combination[0]}_{labware_combination[1]}')
                unoptimized_command_line = print_command(np.array(range(jobs.shape[0])),jobs,f'source{labware_combination[0]}', f'dest{labware_combination[1]}')
                np.savetxt(f'analysis/cost_analysis/unoptimized/{labware_combination[0]}_{labware_combination[1]}/unoptimized_command_line_cost_{non_optimized_distance}_s{labware_combination[0]}_d{labware_combination[1]}_r{r}.csv',
                           unoptimized_command_line,fmt='%s',delimiter=',')
        

        if not os.path.exists(f'analysis/cost_analysis/unoptimized_command_lines/{labware_combination[0]}_{labware_combination[1]}'):
            os.makedirs(f'analysis/cost_analysis/unoptimized_command_lines/{labware_combination[0]}_{labware_combination[1]}')

        names = []
        for filename in glob.glob(f'analysis/cost_analysis/unoptimized/{labware_combination[0]}_{labware_combination[1]}/*.csv'):
            # get the name of the file without the path
            filename = os.path.basename(filename)
            names.append(filename)
        # creat a dataframe to store the data
        names_df = pd.DataFrame(names, columns=['filename'])
        # save the dataframe to a csv file
        names_df.to_csv(f'analysis/cost_analysis/unoptimized/{labware_combination[0]}_{labware_combination[1]}/filenames_mapping.csv', index=True)
        for i in range(len(names_df)):
            # copy the file to the command_lines folder
            shutil.copy(f'analysis/cost_analysis/unoptimized/{labware_combination[0]}_{labware_combination[1]}/{names_df.iloc[i,0]}', f'analysis/cost_analysis/unoptimized_command_lines/{labware_combination[0]}_{labware_combination[1]}/{i}.csv')
            # rename the file as the index of the dataframe
        

        if not os.path.exists(f'analysis/cost_analysis/optimized_command_lines/{labware_combination[0]}_{labware_combination[1]}'):
            os.makedirs(f'analysis/cost_analysis/optimized_command_lines/{labware_combination[0]}_{labware_combination[1]}')
        names = []
        for filename in glob.glob(f'analysis/cost_analysis/optimized/{labware_combination[0]}_{labware_combination[1]}/*.csv'):
            # get the name of the file without the path
            filename = os.path.basename(filename)
            names.append(filename)
        # creat a dataframe to store the data
        names_df = pd.DataFrame(names, columns=['filename'])
        # save the dataframe to a csv file
        names_df.to_csv(f'analysis/cost_analysis/optimized/{labware_combination[0]}_{labware_combination[1]}/filenames_mapping.csv', index=True)
        for i in range(len(names_df)):
            # copy the file to the command_lines folder
            shutil.copy(f'analysis/cost_analysis/optimized/{labware_combination[0]}_{labware_combination[1]}/{names_df.iloc[i,0]}', f'analysis/cost_analysis/optimized_command_lines/{labware_combination[0]}_{labware_combination[1]}/{i}.csv')


labware_combination: [12, 12]
repeat: 0
num_candidates=1


NameError: name 'os' is not defined